# Generate Clean Dataset

The clean dataset is a modernization of the CATSS Database, presented with minimal changes, in UTF8, exported as JSON.

In [71]:
import re
from pathlib import Path
from greekutils import beta2unicode # do: pip install greek-utils==0.2

data = Path('../source/')

In [122]:
# CCAT transcription to UTF8
# Greek to be handled by greekutils

# Hebrew
# first pattern to match is used
trans2utf8 = [
    (r'\)', 'א'),
    (r'B', 'ב'),
    (r'G', 'ג'),
    (r'D', 'ד'),
    (r'H', 'ה'),
    (r'W', 'ו'),
    (r'Z', 'ז'),
    (r'X', 'ח'),
    (r'\+', 'ט'),
    (r'Y', 'י'),
    (r'.*K\b', 'ך'),
    (r'K', 'כ'),
    (r'L', 'ל'),
    (r'.*M\b', 'ם'),
    (r'M', 'מ'),
    (r'.*N\b', 'ן'),
    (r'N', 'נ'),
    (r'S', 'ס'),
    (r'\(', 'ע'),
    (r'.*P\b', 'ף'),
    (r'P', 'פ'),
    (r'.*C\b', 'ץ'),
    (r'C', 'צ'),
    (r'Q', 'ק'),
    (r'R', 'ר'),
    (r'&', 'שׂ'),
    (r'\$', 'שׁ'),
    (r'T', 'ת'),
    (r'-', '־'),
    (r'\\', ''),
    (r' ', ' '),
]

trans2utf8 = [(re.compile(patt), utf8) for patt, utf8 in trans2utf8]

def utf8_hebrew(string):
    """Convert transcribed Hebrew to UTF8 Hebrew"""
    utf8_string = ''
    for i, c in enumerate(string):
        for patt, utf8 in trans2utf8:
            if patt.match(string, i, i+1): # match at specific position
                utf8_string += utf8
                break
    return utf8_string

Demonstrate the functions.

In [123]:
beta2unicode.convert('KAI\ A)KATASKEU/ASTOS')

'καὶ ἀκατασκεύαστοσ'

In [124]:
utf8_hebrew('HYTH $M B\BYT-LXM')

'היתה שׁם בבית־לחם'

In [108]:
test = re.compile(r'.*K\b')

test.match('PARK ISK', 3, 4)

<re.Match object; span=(3, 4), match='K'>

## Exploration

In [7]:
file2text = {}

for file in data.glob('*.par'):
    file2text[file.stem] = file.read_text().strip().split('\n')